In [4]:
!pip install pandas



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\dusti\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
#Filtern der relevanten Daten also Szeanrien A1-A3/A1-A4/Cradle to Gate
#danach löschen aller dateien mit Einträgen=0

import pandas as pd

# Schritt 1: Datei laden (Pfad ggf. anpassen)
df = pd.read_csv(r"c:\Users\dusti\Desktop\KI-Futter\Schritt6.1.-DataFrame-ExcelTabellen\GesamteDaten_als_DataFrame.csv")

# Filtern der relevanten Szenarien (case-insensitive)
szenarien_keywords = ["a1-a3", "a1-a4", "cradle to gate"]
daten_szenarien_A = df[df["Datei"].str.lower().str.contains('|'.join(szenarien_keywords))]

# NaN entfernen
daten_szenarien_A = daten_szenarien_A.dropna()

# Spannweite in numerisch umwandeln und NaNs entfernen
daten_szenarien_A["Spannweite"] = pd.to_numeric(daten_szenarien_A["Spannweite"], errors="coerce")
daten_szenarien_A = daten_szenarien_A.dropna(subset=["Spannweite"])

# Leerzeichen in "Deckensystem" entfernen (Trim)
if "Deckensystem" in daten_szenarien_A.columns:
    daten_szenarien_A["Deckensystem"] = daten_szenarien_A["Deckensystem"].str.strip()

# Ergebnis speichern
daten_szenarien_A.to_excel(r"c:\Users\dusti\Desktop\KI-Futter\Schritt6.2.-DataFrame-ExcelTabellen\Daten_Szenario_A.xlsx", index=False)

# Ausgabe des DataFrames
print(daten_szenarien_A)

#Händische Überarbeitung der Datei- Filtern der Szenarien mit C+D und Ausschluss sowie Anpassen der Datenquelle 8 an Referenzfluss kg CO2-äq./m2
#c:\Users\dusti\Desktop\KI-Futter\Schritt6.-DataFrame-ExcelTabellen\Daten_Szenario_A_Struktur.xlsx- neue strukturierte Datei
#Umwandeln der Datei in überabeiteten DataFrame

df_überarbeitet = pd.read_excel(r"c:\Users\dusti\Desktop\KI-Futter\Schritt6.1.-DataFrame-ExcelTabellen\Daten_Szenario_A_Struktur.xlsx")

                      Deckensystem    GWP  Spannweite Nutzlast  \
15            Stahlbetonflachdecke   58.6         6.0      3.5   
16                 Hohlkörperdecke   45.7         6.0      3.5   
17             Spannbetonhohldiele   40.3         6.0      3.5   
18                     Rippendecke   42.9         6.0      3.5   
19   Holz-Beton-Verbundrippendecke  -35.9         6.0      3.5   
..                             ...    ...         ...      ...   
536             Tramtraversendecke  -21.0         5.0        3   
537             Tramtraversendecke  150.0         9.0        3   
542         Stahlbetonverbunddecke  180.7         3.0        3   
543         Stahlbetonverbunddecke  330.0         5.0        3   
544         Stahlbetonverbunddecke  725.0         9.0        3   

                                         Datei  
15   Datenquelle 14_A1-A3(Referenzgebäude).txt  
16   Datenquelle 14_A1-A3(Referenzgebäude).txt  
17   Datenquelle 14_A1-A3(Referenzgebäude).txt  
18   Datenq

In [6]:
#Hinzufügen der generierten Datenbasis in DataFrame
excel_generiert=(r"c:\Users\dusti\Desktop\KI-Futter\Schritt6.2.-DataFrame-ExcelTabellen\Selfmade-Datenbasis-Update.xlsx")
all_sheets=pd.read_excel(excel_generiert, sheet_name=None)

relevante_spalten=["Deckensystem","Nutzlast","Spannweite","GWP"]

gefilterte_dfs=[]

for name, df in all_sheets.items():
    vorhandene_spalten=[spalte for spalte in relevante_spalten if spalte in df.columns]
    df_gefiltert=df[vorhandene_spalten].copy()

    df_gefiltert["Datei"] = "Generierte Daten"

    gefilterte_dfs.append(df_gefiltert)

df_gesamt= pd.concat(gefilterte_dfs, ignore_index=True)    

df_literatur_generiert = pd.concat([df_gesamt,df_überarbeitet], ignore_index=True)

print(df_literatur_generiert)
#Speichern des dataFrames für Tool in lokalem Ordner
ausgabe_pfad = r"c:\Users\dusti\Desktop\KI-Futter\Schritt6.2.-DataFrame-ExcelTabellen\Kombinierte_Datenbasis.xlsx"

df_literatur_generiert.to_excel(ausgabe_pfad, index=False)

               Deckensystem  Nutzlast  Spannweite        GWP  \
0      Stahlbetonflachdecke       1.5         3.0  40.053200   
1      Stahlbetonflachdecke       3.0         3.0  40.053200   
2      Stahlbetonflachdecke       5.0         3.0  40.053200   
3      Stahlbetonflachdecke       7.0         3.0  40.823840   
4      Stahlbetonflachdecke       8.5         3.0  41.594480   
..                      ...       ...         ...        ...   
115      Tramtraversendecke       3.0         5.0  -4.200000   
116      Tramtraversendecke       3.0         9.0  16.666667   
117  Stahlbetonverbunddecke       3.0         3.0  60.233333   
118  Stahlbetonverbunddecke       3.0         5.0  66.000000   
119  Stahlbetonverbunddecke       3.0         9.0  80.555556   

                       Datei  
0           Generierte Daten  
1           Generierte Daten  
2           Generierte Daten  
3           Generierte Daten  
4           Generierte Daten  
..                       ...  
115  Datenquel

In [7]:
# Trainingsmodell Lineare Regression der Deckensysteme anhand der Eingabeparameter Nutzlast und Spannweite
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np

all_predictions=[]
# Schritt 1: Daten laden
df = pd.read_excel(r"c:\Users\dusti\Desktop\KI-Futter\Schritt6.2.-DataFrame-ExcelTabellen\Kombinierte_Datenbasis.xlsx")

df["Spannweite"] = pd.to_numeric(df["Spannweite"], errors="coerce")
df["Nutzlast"] = pd.to_numeric(df["Nutzlast"], errors="coerce")
df["GWP"] = pd.to_numeric(df["GWP"], errors="coerce")

# Daten bereinigen aufgrund keiner Abbildung von Extremtragwerken
df = df[
    (df["Spannweite"] >= 3) & (df["Spannweite"] <= 12) &
    (df["Nutzlast"] >= 1) & (df["Nutzlast"] <= 10)]

deckensysteme = df["Deckensystem"].unique()
#Iteration über jedes Deckensystem 
for system in deckensysteme:
    teil_df = df[df["Deckensystem"] == system]

    if len(teil_df) < 3:
        print(f"❗ Zu wenige Daten für '{system}' – übersprungen.\n")
        continue
    # Trainingsdaten definieren 
    X = teil_df[["Spannweite", "Nutzlast"]]
    y = teil_df["GWP"]
# Modell erstellen und trainieren
    model = LinearRegression()
    model.fit(X, y)
# Vorhersageraster definieren 
    new_spannweiten = np.arange(3, 13, 1)
    new_nutzlasten = np.arange(1, 11, 1)
    new_data = np.array([[s, n] for s in new_spannweiten for n in new_nutzlasten])
    
    new_df = pd.DataFrame(new_data, columns=["Spannweite", "Nutzlast"])
    new_predictions = model.predict(new_df)
    
    new_df["GWP_predicted"] = new_predictions
    new_df["Deckensystem"] = system
#Vorhersagen dem neuen DataFrame hinzufügen
    all_predictions.append(new_df)

# Alle Ergebnisse zusammenfügen
result_df = pd.concat(all_predictions, ignore_index=True)

# Als Excel speichern
output_path = r"C:\Users\dusti\Desktop\KI-Futter\Schritt6.2.-DataFrame-ExcelTabellen\Predictions_Lineare_Regression_Literatur_generiert.xlsx"
result_df.to_excel(output_path, index=False)

❗ Zu wenige Daten für 'Hohlkörperdecke' – übersprungen.

❗ Zu wenige Daten für 'Rippendecke' – übersprungen.

❗ Zu wenige Daten für 'Kappendecke doppelt gespannt' – übersprungen.

❗ Zu wenige Daten für 'Kappendecke einfach gespannt' – übersprungen.



In [8]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Gewichtungsfaktoren
GENERIERTE_DATEN_FAKTOR_STB = 10  # Stahlbetonflachdecke generierte Daten
GENERIERTE_DATEN_FAKTOR_ANDERE = 5  # Andere generierte Daten
LITERATUR_FAKTOR = 1  # Für alle Literaturdaten

all_predictions = []

# Daten laden
df = pd.read_excel(r"c:\Users\dusti\Desktop\KI-Futter\Schritt6.2.-DataFrame-ExcelTabellen\Kombinierte_Datenbasis.xlsx")

# Numerische Umwandlung
df["Spannweite"] = pd.to_numeric(df["Spannweite"], errors="coerce")
df["Nutzlast"] = pd.to_numeric(df["Nutzlast"], errors="coerce")
df["GWP"] = pd.to_numeric(df["GWP"], errors="coerce")

# Daten filtern
df = df[
    (df["Spannweite"] >= 3) & (df["Spannweite"] <= 12) &
    (df["Nutzlast"] >= 1) & (df["Nutzlast"] <= 10)
]

deckensysteme = df["Deckensystem"].unique()

for system in deckensysteme:
    teil_df = df[df["Deckensystem"] == system]

    if len(teil_df) < 3:
        print(f"❗ Zu wenige Daten für '{system}' – übersprungen.\n")
        continue

    X = teil_df[["Spannweite", "Nutzlast"]]
    y = teil_df["GWP"]

    if "Datei" in teil_df.columns:
        if system == "Stahlbetonflachdecke":
            weights = np.where(
                teil_df["Datei"].str.lower() == "generierte daten",
                GENERIERTE_DATEN_FAKTOR_STB,
                LITERATUR_FAKTOR
            )
        else:
            weights = np.where(
                teil_df["Datei"].str.lower() == "generierte daten",
                GENERIERTE_DATEN_FAKTOR_ANDERE,
                LITERATUR_FAKTOR
            )
    else:
        weights = None

    model = LinearRegression()
    if weights is not None:
        model.fit(X, y, sample_weight=weights)
    else:
        model.fit(X, y)

    # Vorhersageraster definieren
    new_spannweiten = np.arange(3, 13, 1)
    new_nutzlasten = np.arange(1, 11, 1)
    new_data = np.array([[s, n] for s in new_spannweiten for n in new_nutzlasten])

    new_df = pd.DataFrame(new_data, columns=["Spannweite", "Nutzlast"])
    new_predictions = model.predict(new_df)

    new_df["GWP_predicted"] = new_predictions
    new_df["Deckensystem"] = system

    all_predictions.append(new_df)

result_df = pd.concat(all_predictions, ignore_index=True)

output_path = r"C:\Users\dusti\Desktop\KI-Futter\Schritt6.2.-DataFrame-ExcelTabellen\Predictions_Lineare_Regression_Gewichtet.xlsx"
result_df.to_excel(output_path, index=False)

print(f"Vorhersagen gespeichert: {output_path}")


❗ Zu wenige Daten für 'Hohlkörperdecke' – übersprungen.

❗ Zu wenige Daten für 'Rippendecke' – übersprungen.

❗ Zu wenige Daten für 'Kappendecke doppelt gespannt' – übersprungen.

❗ Zu wenige Daten für 'Kappendecke einfach gespannt' – übersprungen.

Vorhersagen gespeichert: C:\Users\dusti\Desktop\KI-Futter\Schritt6.2.-DataFrame-ExcelTabellen\Predictions_Lineare_Regression_Gewichtet.xlsx
